# Import

In [10]:
from ultralytics import YOLO

# model

In [ ]:
model = YOLO("./model/yolo11s.pt")
data_path = "./data/data.yaml"

# Predict

In [14]:
model.train(data=data_path,epochs=100,patience=10)

Ultralytics 8.3.162  Python-3.12.10 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=./model/yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose=12.0, pretra

RuntimeError: Dataset 'data/data.yaml' error  Dataset 'data/data.yaml' images not found, missing path 'C:\Users\user\OneDrive\ \\3 \data\val\images'
Note dataset download directory is 'C:\Users\user\OneDrive\ \\3 \datasets'. You can update this in 'C:\Users\user\AppData\Roaming\Ultralytics\settings.json'